**Tarea** 3

In [ ]:
##Modulos

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

In [ ]:
## Clases

################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)


**Parte 1**

Movimiento Browniano con Pandas

In [ ]:
def bm_2d(pasos=1000, velocidad=6, posicionIni=[0,0]):
  #referencia  de clase Vec2d
  velocity = Vec2d(velocidad,0)
  ## Estructura pandas dataframe
  BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':posicionIni[0],'y_pos':posicionIni[1]}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  #Generacion de direcciones aleatorias
  for i in range(pasos-1):
    #es decir de -pi a pi posibilidad de giros
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity=velocity.rotated(turn_angle)

    temp_df= pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x,'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df,temp_df], ignore_index=True)
  return BM_2d_df

*CRW con pandas*

In [ ]:
def CRW_2d(pasos=1000, velocidad=6,posicionIni=[0,0],factorCauchy=0.4,):
  velocity = Vec2d(velocidad,0)

  #estructura DataFRame pandas
  CRW_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':posicionIni[0],'y_pos':posicionIni[1]}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
  ListaPDTrayectorias = []
  distribucionDireccion = wrapcauchy(factorCauchy,0,scale=1)
  listaDirecciones = distribucionDireccion.rvs(size=pasos)
  for i in range(0, pasos-1):
    turn_angle = listaDirecciones[i]
    velocity=velocity.rotated(turn_angle)
    temp_df= pd.DataFrame([{'x_pos': CRW_2d_df['x_pos'].iloc[-1] + velocity.x, 'y_pos': CRW_2d_df['y_pos'].iloc[-1] + velocity.y}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
  return CRW_2d_df

  ## temp_df= pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i]+velocity.x,'y_pos':CRW_2d.df.y_pos[i]+velocity.y}])

Funcion aux de calculo de distancia entre 2 puntos en el plano

In [ ]:
def distanciaXY(x1=0,y1=0,x2=1,y2=1):
  distancia = math.sqrt((x2-x1)**2+(y2-y1)**2)
  return distancia

calculo path lenght de trayectoria

In [ ]:
def trajectoryLenght(trayectoria):
  distanciaRetorno = 0.0
  pasos=trayectoria.shape[0]
  #Print(pasos)
  # Almacena Z distancia recorrida al punto de trayecto
  TrajectoryLenghtAC = pd.DataFrame(columns=['x_pos','y_pos','z_pos'])
  temp_df = pd.DataFrame([{'x_pos':0,'y_pos':0,'z_pos':0}])
  TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  for i in range(1,pasos):
    distanciaRetorno=distanciaRetorno + distanciaXY(trayectoria.x_pos[i-1],trayectoria.x_pos[i],trayectoria.y_pos[i])
    temp_df= pd.DataFrame([{'x_pos':trayectoria.x_pos[i], 'y_pos':trayectoria.y_pos[i], 'z_pos':distanciaRetorno}])
    TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  return TrajectoryLenghtAC




Prueba 1

In [ ]:
BM_2d_df = bm_2d(2000,3,[0,0])

fig_BM_2d=go.Figure()

fig_BM_2d.add_trace(go.Scatter(
    x=BM_2d_df.x_pos,
    y=BM_2d_df.y_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name= "Browniano",
    showlegend=True
))

Prueba 1 CRW

In [ ]:
CRW_2d_df = CRW_2d(2000, 3,[0, 0], 0.4)

fig_BM_2d=go.Figure()

fig_BM_2d.add_trace(go.Scatter(
    x=CRW_2d_df.x_pos,
    y=CRW_2d_df.y_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name="Correlated Walk",
    showlegend=True
))

Path Lenght

In [ ]:
BM_2d_df_3 = bm_2d(2000,3,[0,0])
BM_2d_df_6 = bm_2d(2000,6,[0,0])
CRW_2d_df_6 = CRW_2d(2000,6,[0,0],0.4)
fig_BM_2d=go.Figure()
PathLenght=trajectoryLenght(BM_2d_df_3)
fig_BM_2d.add_trace(go.Scatter(
    x=PathLenght.x_pos.index,
    y=PathLenght.z_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name="Movimiento Browniano Velocidad 3",
    showlegend=True
))
PathLenght=trajectoryLenght(BM_2d_df_6)
fig_BM_2d.add_trace(go.Scatter(
    x=PathLenght.x_pos.index,
    y=PathLenght.z_pos,
    marker=dict(size=2),
    line=dict(width=6),
    mode='lines',
    name="BM Walk Lenght velocidad 6",
    showlegend=True
))
PathLenght=trajectoryLenght(CRW_2d_df_6)
fig_BM_2d.add_trace(go.Scatter(
    x=PathLenght.x_pos.index,
    y=PathLenght.z_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name='CRW Walk Lenght velocidad 6',
    showlegend=True
))
fig_BM_2d.show()

Parte 2 Mean Squared Displacement - (BM vs CRW)

In [ ]:
def distanciaXYmean(x1=0,y1=0,x2=1,y2=1):
  distancia = math.sqrt((x2-x1)**2+(y2-y1)**2)

  return distancia

MSD Mean squared displacement

In [ ]:
def trajectoryMean(trayectoria):
  distanciaRetorno = 0.0
  pasos=trayectoria.shape[0]
  TrajectoryLenghtAC = pd.DataFrame(columns=['x_pos','y_pos','z_pos'])
  temp_df = pd.DataFrame([{'x_pos':0,'y_pos':0,'z_pos':0}])
  TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  for i in range(1,pasos-1):
    contadorProm=0
    distanciaRetorno = 0.0
    for ii in range(0,pasos-i):
      contadorProm=contadorProm+1
      # si (ii+i) < (pasos-1)
      distanciaRetorno=distanciaRetorno + (distanciaXYmean(trayectoria.x_pos[ii],trayectoria.y_pos[ii],trayectoria.x_pos[ii+i],trayectoria.y_pos[ii+i]))**2
    distanciaRetorno=(distanciaRetorno / contadorProm)
    #Print ('i: '+str(i)+' ii: 'str(ii)+' distanciaRetorno: 'str(distanciaRetorno))
    Temp_df= pd.DataFrame([{'x_pos':trayectoria.x_pos[i-1], 'y_pos':trayectoria.y_pos[i-1], 'z_pos':distanciaRetorno}])
  return TrajectoryLenghtAC



Mostrar Brownian motion y Corelated Random Walk

In [ ]:
BM_2d_df_6_2 = bm_2d(1000,6,[0,0])
CRW_2d_df_6_2 = CRW_2d(1000,6,[0,0],0.4)
BM_2d_df_6_mean2 = trajectoryMean(BM_2d_df_6_2)
CRW_2d_df_9_mean2 = trajectoryMean(CRW_2d_df_6_2)
fig_BM_2d=go.Figure()
fig_BM_2d.add_trace(go.Scatter(
    x=BM_2d_df_6_mean2.x_pos.index,
    y=BM_2d_df_6_mean2.z_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name="Brownian motion y MSD",
    showlegend=True
))

fig_BM_2d.add_trace(go.Scatter(
    x=CRW_2d_df_9_mean2.x_pos.index,
    y=CRW_2d_df_9_mean2.z_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name='CRW Walk MSD',
    showlegend=True
))

fig_BM_2d.show()

## Parte 3 Turning - angle Distribución

In [ ]:
# Cálculo de ángulo

def turning_angle_nivpfer(pos_a, pos_b, pos_c):
  vec_ab = np.array(pos_a)-np.array(pos_b)
  norm_ab = np.linalg.norm(vec_ab)
  vec_bc = np.array(pos_b)-np.array(pos_c)
  norm_bc = np.linalg.norm(vec_bc)
  dot_p = sum(vec_ab*vec_bc)
  cos_theta=dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

  cross_p = np.cross(vec_ab,vec_bc)
  orient = np.sign(cross_p)
  if orient == 0:
      orient = 1

  theta = np.arccos(np.around(cos_theta,4)) * orient
  return theta

cargar crw

In [ ]:
# Load Existing trajectories
# CRW velocidad = 6
# Wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

#Load existing trajectories
#CRW velocidad = 6
# Wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

## Cálculo de ángulos en trayectoria archivo crw_6_9.csv y trayectoria crw_6_6.csv

In [ ]:
# extraer cantidad de registros
pasos=CRW_2d_df_9.shape[0]

#crear estructura de pandas dataframe
TrayectoryAndTA_9 = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos'])
temp_df = pd.DataFrame([{'x_pos':CRW_2d_df_9.x_pos[0],'y_pos':CRW_2d_df_9.y_pos[0],'z_pos':0}])
TrayectoryAndTA_9 = pd.concat([TrayectoryAndTA_9, temp_df], ignore_index=True)

#calcular angulos
for i in range(1, pasos-1):
  angulo=turning_angle_nivpfer(CRW_2d_df_9.iloc[i-1],CRW_2d_df_9.iloc[i],CRW_2d_df_9.iloc[i+1])
  temp_df= pd.DataFrame([{'x_pos':CRW_2d_df_9.x_pos[i],'y_pos':CRW_2d_df_9.y_pos[i], 'z_pos':angulo}])
  TrayectoryAndTA_9 = pd.concat([TrayectoryAndTA_9, temp_df], ignore_index=True)

  pasos=CRW_2d_df_6.shape[0]
  TrayectoryAndTA_6 = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos'])
  temp_df = pd.DataFrame([{'x_pos':CRW_2d_df_6.x_pos[0], 'y_pos':CRW_2d_df_6.y_pos[0],'z_pos':0}])
  TrayectoryAndTA_6 = pd.concat([TrayectoryAndTA_6, temp_df], ignore_index=True)

for i in range(1, pasos-1):
  angulo=turning_angle_nivpfer(CRW_2d_df_6.iloc[i-1],CRW_2d_df_6.iloc[i],CRW_2d_df_6.iloc[i+1])
  temp_df= pd.DataFrame([{'x_pos':CRW_2d_df_6.x_pos[i],'y_pos':CRW_2d_df_6.y_pos[i],'z_pos':angulo}])
  TrayectoryAndTA_6 = pd.concat([TrayectoryAndTA_6, temp_df], ignore_index=True)

In [ ]:
# Calculo de PDF

def depuraCauchyPDF(aregloPDF):
  tamano_arreglo=int(aregloPDF.size/2)
  centro_real=0
  contador=0
  for i in range(tamano_arreglo,aregloPDF.size):
    if aregloPDF[(tamano_arreglo-1)+contador] > 0:
      centro_real=contador
      i=aregloPDF.size
    else:
      contador=contador+1

  for i in range(0,centro_real):
    aregloPDF[tamano_arreglo+i]=aregloPDF[tamano_arreglo+centro_real]

  for i in range(0,tamano_arreglo-centro_real):
    aregloPDF[tamano_arreglo-i]=(aregloPDF[(tamano_arreglo-1)+(i+centro_real)])

  return aregloPDF

In [ ]:
# Velocidad 6  para correlated random walk
# 0.9 para Wrapcauchy

miu = 0.0
distribucionDireccion = wrapcauchy(0.9,loc=0, scale=1,)

aux_domain_9 = np.linspace(TrayectoryAndTA_9.z_pos.min(), TrayectoryAndTA_9.z_pos.max(), 1000)
cauchy_pdf = np.array([distribucionDireccion.pdf(i)for i in aux_domain_9])
cauchy_pdf_depurado_9=depuraCauchyPDF(cauchy_pdf)

distribucionDireccion = wrapcauchy(0.6,loc=0,scale=1,)
aux_domain_6 = np.linspace(TrayectoryAndTA_6.z_pos.min(), TrayectoryAndTA_6.z_pos.max(), 1000)
cauchy_pdf = np.array([distribucionDireccion.pdf(i)for i in aux_domain_6])
cauchy_pdf_depurado_6=depuraCauchyPDF(cauchy_pdf)

fig = go.Figure(data=[go.Histogram(x=TrayectoryAndTA_9.z_pos,
                                   histnorm='probability density',
                                   name='Observ .9',
                                   marker_color="purple",
                                   )])

fig.add_histogram(x=TrayectoryAndTA_6.z_pos,
                                  histnorm='probability density',
                                  name='Observ .6',
                                  marker_color="pink",
                                  )

fig.add_trace(go.Scatter(
    x=aux_domain_9,
    y=cauchy_pdf_depurado_9,
    marker=dict(size=3),
    line=dict(width=7,color='gray'),
    mode='lines',
    name='CR Walk Cauchy .9',
    showlegend=True
))
fig.add_trace(go.Scatter(
    x=aux_domain_6,
    y=cauchy_pdf_depurado_6,
    marker=dict(size=3),
    line=dict(width=7,color='black'),
    mode='lines',
    name='CR Walk Cauchy .6',
    showlegend=True
))

fig.update_yaxes(
    scaleanchor='x',
    scaleratio=1
)

fig.show()


Parte 4 Step-Lenght DistributioYy######sdfsdffjh11111iiiiiiiii

In [ ]:
# Cargar el archivo Levy 6_1
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Cargar el archivo Levy 6_7
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# GENERAR LEVY FLIGHTS

def LevyFlyPd(pasos, speed, miu, levyExponent, levyBeta, factorCauchy):
  s_pos = [0,0]
  velocity = Vec2d(speed,0)

  #crear instancia de objeto wrapcauchy
  distribucionDireccion = wrapcauchy(0.4,0,scale=1)

  #Obtener giros de la trayectoria
  listaDirecciones = distribucionDireccion.rvs(size=pasos)

  #crear instancia de objeto levy
  distribucionPasos=levy_stable(alpha=levyExponent, beta=levyBeta, loc=miu)

  #obtener pasos para trayectoria
  listaCantidadPasos=distribucionPasos.rvs(size=pasos)

  #creación de dataFrames Pandas
  pdLevy = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp3d = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])
  pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
  PasosPendientes=0
  IndicePasos = 0
  IndiceCambiosDireccion = 0

  for i in range(0,pasos-1):
    if PasosPendientes < 1 :
      velocity=velocity.rotated(listaDirecciones[IndiceCambiosDireccion])
      IndiceCambiosDireccion=IndiceCambiosDireccion + 1
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x, 'y_pos':pdLevy.y_pos[i]+velocity.y}])
      pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
      PasosPendientes=int(listaCantidadPasos[IndicePasos])
      IndicePasos=IndicePasos+1

    else:
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y}])
      pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
      PasosPendientes=PasosPendientes-1

  return pdLevy




In [ ]:
# Funcion que calcula stepLenght

def calculaStepLenght(trayectoria):
  trayectoriaConGiro = pd.DataFrame(columns=['x_pos','y_pos', 'z_angulo'])
  temp_df = pd.DataFrame([{'x_pos':trayectoria.x_pos[0], 'y_pos':trayectoria.y_pos[0],'z_angulo':0}])
  trayectoriaConGiro = pd.concat([trayectoriaConGiro, temp_df], ignore_index=True)
  for i in range(1, trayectoria.shape[0]-1):
    angulo=turning_angle_nivpfer(trayectoria.iloc[i-1],trayectoria.iloc[i],trayectoria.iloc[i+1])
    temp_df= pd.DataFrame([{'x_pos':trayectoria.x_pos[i],'y_pos':trayectoria.y_pos[i], 'z_angulo':angulo}])
    trayectoriaConGiro = pd.concat([trayectoriaConGiro, temp_df], ignore_index=True)

  trayectoriaConGiroYPasos = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_steps'])
  temp_df = pd.DataFrame([{'x_pos':trayectoriaConGiro.x_pos[0],'y_pos':trayectoriaConGiro.y_pos[0],'z_angulo':0,'z_steps':1}])
  trayectoriaConGiroYPasos = pd.concat([trayectoriaConGiroYPasos, temp_df], ignore_index=True)
  Xarista=trayectoriaConGiro.x_pos[0]
  Yarista=trayectoriaConGiro.y_pos[0]
  pasos=1
  for i in range(0, trayectoriaConGiro.shape[0]-1):
    if trayectoriaConGiro.z_angulo[i]==0:
      pasos=pasos+1
    else:
      temp_df = pd.DataFrame([{'x_pos':trayectoriaConGiro.x_pos[i],'y_pos':trayectoriaConGiro.y_pos[i],'z_angulo':trayectoriaConGiro.z_angulo[i],'z_steps':pasos}])
      trayectoriaConGiroYPasos = pd.concat([trayectoriaConGiroYPasos, temp_df], ignore_index=True)
      Xarista=trayectoriaConGiro.x_pos[i]
      Yarista=trayectoriaConGiro.y_pos[i]
      pasos=1

  return trayectoriaConGiroYPasos

In [ ]:
# Definir velocidad

velocidad = 6

#crear dataframe para trayectoria factor 1
TrayectoriaLF10 = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos'])
temp_df = pd.DataFrame([{'x_pos':Levy_2d_df_1.x_pos[0],'y_pos':Levy_2d_df_1.y_pos[0],'z_pos': 0}])
TrayectoriaLF10 = pd.concat([TrayectoriaLF10, temp_df], ignore_index=True)
for i in range(1, Levy_2d_df_1.shape[0]-1):
  angulo= turning_angle_nivpfer(Levy_2d_df_1.iloc[i-1], Levy_2d_df_1.iloc[i],Levy_2d_df_1.iloc[i+1])
  temp_df= pd.DataFrame([{'x_pos':Levy_2d_df_1.x_pos[i],'y_pos':Levy_2d_df_1.y_pos[i],'z_pos':angulo}])
  TrayectoriaLF10 = pd.concat([TrayectoriaLF10, temp_df], ignore_index=True)

velocidad = 6
TrayectoriaLF07 = pd.DataFrame(columns=['x_pos','y_pos','z_pos'])
temp_df = pd.DataFrame([{'x_pos':Levy_2d_df_7.x_pos[0],'y_pos':Levy_2d_df_7.y_pos[0],'z_pos':0}])
TrayectoriaLF07 = pd.concat([TrayectoriaLF07, temp_df], ignore_index=True)
for i in range(1, Levy_2d_df_7.shape[0]-1):
  angulo=turning_angle_nivpfer(Levy_2d_df_7.iloc[i-1],Levy_2d_df_7.iloc[i],Levy_2d_df_7.iloc[i+1])
  temp_df= pd.DataFrame([{'x_pos':Levy_2d_df_7.x_pos[i],'y_pos':Levy_2d_df_7.y_pos[i],'z_pos':angulo}])
  TrayectoriaLF07 = pd.concat([TrayectoriaLF07, temp_df], ignore_index=True)

In [ ]:
sinLineasTrayectoriaLF10 = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_angulo', 'z_steps'])
temp_df = pd.DataFrame([{'x_pos':TrayectoriaLF10.x_pos[0], 'y_pos': TrayectoriaLF10.y_pos[0], 'z_angulo':0,'z_steps':1}])
sinLineasTrayectoriaLF10 = pd.concat([sinLineasTrayectoriaLF10, temp_df], ignore_index=True)
Xarista=TrayectoriaLF10.x_pos[0]
Yarista=TrayectoriaLF10.y_pos[0]
pasos=1
for i in range(1, TrayectoriaLF10.shape[0]-1):
  if TrayectoriaLF10.z_pos[i]==0:
    pasos=pasos+1
  else:
    temp_df = pd.DataFrame([{'x_pos':TrayectoriaLF10.x_pos[i],'y_pos':TrayectoriaLF10.y_pos[i],'z_angulo':TrayectoriaLF10.z_pos[i],'z_steps':pasos}])
    sinLineasTrayectoriaLF10 = pd.concat([sinLineasTrayectoriaLF10, temp_df], ignore_index=True)
    Xarista=TrayectoriaLF10.x_pos[i]
    Yarista=TrayectoriaLF10.y_pos[i]
    pasos=1

#se realiza lo mismo para .7
sinLineasTrayectoriaLF07 = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_angulo', 'z_steps'])
temp_df = pd.DataFrame([{'x_pos':TrayectoriaLF07.x_pos[0], 'y_pos': TrayectoriaLF07.y_pos[0], 'z_angulo':0,'z_steps':1}])
sinLineasTrayectoriaLF07 = pd.concat([sinLineasTrayectoriaLF07, temp_df], ignore_index=True)
Xarista=TrayectoriaLF07.x_pos[0]
Yarista=TrayectoriaLF07.y_pos[0]
pasos=1
for i in range(1, TrayectoriaLF07.shape[0]-1):
  if TrayectoriaLF07.z_pos[i]==0:
    pasos=pasos+1
  else:
    temp_df = pd.DataFrame([{'x_pos':TrayectoriaLF07.x_pos[i],'y_pos':TrayectoriaLF07.y_pos[i],'z_angulo':TrayectoriaLF07.z_pos[i],'z_steps':pasos}])
    sinLineasTrayectoriaLF07 = pd.concat([sinLineasTrayectoriaLF07, temp_df], ignore_index=True)
    Xarista=TrayectoriaLF07.x_pos[i]
    Yarista=TrayectoriaLF07.y_pos[i]
    pasos=1

distribucionPasos=levy_stable(alpha=0.7, beta=1.0, loc=3.0)
aux_domain_lv07 = np.linspace(0, 100, 1000)

distribucionPasos=levy_stable(alpha=0.1, beta=1.0, loc=3.0)
aux_domain_lv10 = np.linspace(0, 100, 1000)
levy_pdf10 = np.array([distribucionPasos.pdf(i)for i in aux_domain_lv10])



In [ ]:
# Cálculo de StepLenght y se obtiene step-lenght dist

# primero .7

sinLineasTrayectoriaLF07 = calculaStepLenght(Levy_2d_df_7)
distribucionPasos=levy_stable(alpha=0.7, beta=1.0, loc=3.0)
aux_domain_lv07 = np.linspace(0, 100, 1000)
levy_pdf07 = np.array([distribucionPasos.pdf(i)for i in aux_domain_lv07])

# ahora para 1

sinLineasTrayectoriaLF10=calculaStepLenght(Levy_2d_df_1)
distribucionPasos=levy_stable(alpha=1.0, beta=1.0, loc=3.0)
aux_domain_lv10 = np.linspace(0, 100, 1000)
levy_pdf10 = np.array([distribucionPasos.pdf(i)for i in aux_domain_lv10])

In [46]:
## gráfica distribución origen vs distribución observada (Histograma)

layout = go.Layout(
    yaxis=dict(
        range=[0,.010]
    ),
    xaxis=dict(
        range=[-100, 1200]
    )
)

fig = go.Figure(data=[go.Histogram(x=sinLineasTrayectoriaLF07.z_steps,
                                   histnorm='probability density',
                                   name='observable .7',
                                   marker_color="gray",

                                   )],
                                   layout=layout
                                     )

fig.add_histogram(x=sinLineasTrayectoriaLF10.z_steps,
                    histnorm='probability density',
                    name='observable 1',
                  marker_color="black",
                  )

fig.add_trace(go.Scatter(
    x=aux_domain_lv07,
    y=levy_pdf07,
    marker=dict(size=3),
    line=dict(width=2,color='purple'),
    mode='lines',
    name='Levy PDF .7',
    showlegend=True
))
fig.add_trace(go.Scatter(
    x=aux_domain_lv10,
    y=levy_pdf10,
    marker=dict(size=3),
    line=dict(width=3,color='pink'),
    mode='lines',
    name='Levy PDF 1',
    showlegend=True
))